In [ ]:
from re import search
import yfinance as yf
import pandas as pd
import numpy as np

# 나스닥 상위 40개 종목 리스트
nasdaq_tickers = [
    "AAPL", "MSFT", "NVDA", "GOOGL", "AMZN", "TSLA", "META", "NFLX", "ADBE", "AVGO",
    "AMD", "INTC", "CSCO", "CMCSA", "PEP", "QCOM", "TXN", "AMAT", "COST", "MRVL",
    "ADP", "VRTX", "ISRG", "INTU", "PYPL", "LRCX", "SBUX", "GILD", "ATVI", "REGN",
    "MDLZ", "BKNG", "SNPS", "ADSK", "ORLY", "KLAC", "FTNT", "PANW", "TEAM", "ZS"
]

# CCI 계산 함수 정의
def calculate_cci(data, n=20):
    """
    특정 종목의 데이터를 받아서 CCI 계산 수행.
    """
    data['TP'] = (data['High'] + data['Low'] + data['Close']) / 3  # Typical Price
    data['SMA_TP'] = data['TP'].rolling(window=n).mean()
    data['MAD'] = data['TP'].rolling(window=n).apply(lambda x: np.mean(np.abs(x - x.mean())), raw=True)
    data['CCI'] = (data['TP'] - data['SMA_TP']) / (0.015 * data['MAD'])
    return data

# CCI 상태 라벨링 함수 정의
cci_bins = np.arange(-200, 220, 10)  # 상태 범위
def label_state(value, bins):
    return np.digitize(value, bins, right=True)

# Scope 범위와 라벨 정의
cci_scope_bins = np.arange(-20,20)  # 상태 범위
def label_state_scope(value, bins):
    return np.digitize(value, bins, right=True)

# 모든 종목 데이터를 다운로드하여 결합
data_frames = []
for ticker in nasdaq_tickers:  # 테스트를 위해 5개 종목만 처리
    datay = yf.download(ticker, start="2017-01-01", end="2024-11-24")
    if datay.empty:
        continue
    if isinstance(datay.columns, pd.MultiIndex):
        datay.columns = [col[0] for col in datay.columns.values]
    data_frames.append(datay)
combined_data = pd.concat(data_frames)
combined_data.reset_index(inplace=True)
combined_data.set_index('Date', inplace=True)

# CCI 계산 및 상태 라벨링
data = calculate_cci(combined_data)
data['CCI_State'] = data['CCI'].apply(lambda x: label_state(x, cci_bins))
data['CCI_State_1'] = data['CCI_State'].shift(1)
data['CCI_State_2'] = data['CCI_State'].shift(2)
data['CCI_State_3'] = data['CCI_State'].shift(3)
data['CCI_State_4'] = data['CCI_State'].shift(4)
data['CCI_State_5'] = data['CCI_State'].shift(5)
data['CCI_State_6'] = data['CCI_State'].shift(6)

data['State'] = data['CCI_State'].astype(float).astype(str)
data['Next_State'] = data['CCI_State'].shift(-1).astype(str)
# 2 3 4
data['State_1'] = data['CCI_State_1'].astype(str)
data['State_2'] = data['CCI_State_2'].astype(str)
data['State_3'] = data['CCI_State_3'].astype(str)
data['State_4'] = data['CCI_State_4'].astype(str)
data['State_5'] = data['CCI_State_5'].astype(str)
data['State_6'] = data['CCI_State_6'].astype(str)

# Scope 및 Scope Label 계산
data['scope'] = data['CCI_State'] - data['CCI_State'].shift(1)
data['scope_label'] = data['scope'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)
# 2
data['scope1'] = data['CCI_State_1'] - data['CCI_State_1'].shift(1)
data['scope_label1'] = data['scope1'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

data['scope2'] = data['CCI_State_2'] - data['CCI_State_2'].shift(1)
data['scope_label2'] = data['scope2'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

data["scope3"] = data["CCI_State_3"] - data["CCI_State_3"].shift(1)
data["scope_label3"] = data["scope3"].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

data['scope4'] = data['CCI_State_4'] - data['CCI_State_4'].shift(1)
data['scope_label4'] = data['scope4'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

data['scope5'] = data['CCI_State_5'] - data['CCI_State_5'].shift(1)
data['scope_label5'] = data['scope5'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

data['scope6'] = data['CCI_State_6'] - data['CCI_State_6'].shift(1)
data['scope_label6'] = data['scope6'].apply(lambda x: label_state_scope(x, cci_scope_bins) if pd.notnull(x) else None)

# 상태 전이 및 기울기(Scope_Label) 쌍별 count 계산
transition_counts = data.groupby(['State', 'Next_State', 'scope_label']).size().reset_index(name='actual_count')
transition_counts1 = data.groupby(['State_1', 'Next_State', 'scope_label1']).size().reset_index(name='actual_count')
transition_counts2 = data.groupby(['State_2', 'Next_State', 'scope_label2']).size().reset_index(name='actual_count')
transition_counts3 = data.groupby(['State_3', 'Next_State', 'scope_label3']).size().reset_index(name='actual_count')
transition_counts4 = data.groupby(['State_4', 'Next_State', 'scope_label4']).size().reset_index(name='actual_count')
transition_counts5 = data.groupby(['State_5', 'Next_State', 'scope_label5']).size().reset_index(name='actual_count')
transition_counts6 = data.groupby(['State_6', 'Next_State', 'scope_label6']).size().reset_index(name='actual_count')


matrix = pd.DataFrame(transition_counts)
#2
matrix1 = pd.DataFrame(transition_counts1)
matrix2 = pd.DataFrame(transition_counts2)
matrix3 = pd.DataFrame(transition_counts3)
matrix4 = pd.DataFrame(transition_counts4)
matrix5 = pd.DataFrame(transition_counts5)
matrix6 = pd.DataFrame(transition_counts6)

print(matrix1)
search_State = "0"
search_inc = 0.0
filter_matrix = matrix.loc[(matrix["State"] == search_State) & (matrix["scope_label"] == search_inc)]



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

      State_1 Next_State  scope_label1  actual_count
0         0.0        0.0           0.0            64
1         0.0        0.0           1.0             2
2         0.0        0.0           2.0             5
3         0.0        0.0           3.0             4
4         0.0        0.0           4.0             9
...       ...        ...           ...           ...
20305     9.0        9.0          23.0             7
20306     9.0        9.0          24.0             7
20307     9.0        9.0          25.0             7
20308     9.0        9.0          26.0             3
20309     9.0        9.0          29.0             2

[20310 rows x 4 columns]


In [ ]:
print(matrix)

      State Next_State  scope_label  actual_count
0       0.0        0.0          0.0            88
1       0.0        0.0          1.0             5
2       0.0        0.0          2.0             5
3       0.0        0.0          3.0             8
4       0.0        0.0          4.0            15
...     ...        ...          ...           ...
16415   9.0        9.0         25.0             8
16416   9.0        9.0         26.0            10
16417   9.0        9.0         27.0             2
16418   9.0        9.0         28.0             1
16419   9.0        9.0         29.0             3

[16420 rows x 4 columns]


In [ ]:
filter_matrix = matrix.loc[(matrix["State"] == "0") & (matrix["scope_label"] == 1.0)]
filter_matrix

,State,Next_State,scope_label,actual_count
1,0,0.0,1.0,5
22,0,1.0,1.0,2
97,0,3.0,1.0,1
132,0,5.0,1.0,2


In [ ]:
def Probabilistic_search1(cci, last_cci):
  state = str(label_state(cci, cci_bins))
  last_state = str(label_state(last_cci, cci_bins))
  scope = float(label_state_scope(int(state) - int(last_state), cci_scope_bins))
  search_State = state
  search_inc = scope
  filter_matrix = matrix2.loc[(matrix2["State_2"] == search_State) & (matrix2["scope_label2"] == search_inc)]
  filter_matrix["State_2"] = filter_matrix["State_2"].astype(int)
  filter_matrix["Next_State"] = filter_matrix["Next_State"].astype(float)
  #print(filter_matrix.loc[filter_matrix["Next_State"] > int(state)])
  down = filter_matrix.loc[( filter_matrix["Next_State"] < int(state) ) & (filter_matrix["Next_State"] > int(state) - 2)]["actual_count"].sum()
  up = filter_matrix.loc[(filter_matrix["Next_State"] > int(state)) & (filter_matrix["Next_State"] < int(state) + 2)]["actual_count"].sum()
  #print(f"state : {state}, last_state : {last_state}, scope : {scope}")
  #print(f"down : {down}, up : {up}")
  return down, up

In [ ]:
 Probabilistic_search1(-103, -81)

(0, 0)

In [ ]:
pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 5.6 MB/s eta 0:00:00


In [ ]:


def Probabilistic_search(cci, cci1, cci2, cci3, cci4, cci5, cci6, cci7):
    state = label_state(cci, cci_bins)
    state1 = label_state(cci1, cci_bins)
    state2 = label_state(cci2, cci_bins)
    state3 = label_state(cci3, cci_bins)
    state4 = label_state(cci4, cci_bins)
    state5 = label_state(cci5, cci_bins)
    state6 = label_state(cci6, cci_bins)
    last_state = label_state(cci7, cci_bins)

    # 상태 리스트
    state_list = [state, state1, state2, state3, state4, state5, state6, last_state]
    # 매트릭스 리스트
    matrix_list = [matrix, matrix1, matrix2, matrix3, matrix4, matrix5, matrix6]

    result_updown = []

    for i in range(len(state_list) - 1):

        index = "State"
        if i == 0:
            index = "State"
        elif i == 1:
            index = "State_1"
        elif i == 2:
            index = "State_2"
        elif i == 3:
            index = "State_3"
        elif i == 4:
            index = "State_4"
        elif i == 5:
            index = "State_5"
        elif i == 6:
            index = "State_6"

        scope_index = "State"
        if i == 0:
            scope_index = "scope_label"
        elif i == 1:
            scope_index = "scope_label1"
        elif i == 2:
            scope_index = "scope_label2"
        elif i == 3:
            scope_index = "scope_label3"
        elif i == 4:
            scope_index = "scope_label4"
        elif i == 5:
            scope_index = "scope_label5"
        elif i == 6:
            scope_index = "scope_label6"

        # 현재 상태와 다음 상태
        current_state = state_list[i]
        next_state = state_list[i + 1]
        scope = float(label_state_scope(current_state - next_state, cci_scope_bins))

        # 현재 상태에 대응하는 matrix 선택
        current_matrix = matrix_list[i]

        # 필터링 및 계산
        filter_matrix = current_matrix.loc[
            (current_matrix[index] == str(float(current_state))) &
            (current_matrix[scope_index] == scope)
        ].copy()

        # 데이터 타입 변환
        #filter_matrix[index] = filter_matrix[index].astype(int)
        filter_matrix["Next_State"] = filter_matrix["Next_State"].astype(float)

        # up/down 계산
        down = filter_matrix.loc[
            (filter_matrix["Next_State"] < int(current_state)) &
            (filter_matrix["Next_State"] > int(current_state) - 2)
        ]["actual_count"].sum()

        up = filter_matrix.loc[
            (filter_matrix["Next_State"] > int(current_state)) &
            (filter_matrix["Next_State"] < int(current_state) + 2)
        ]["actual_count"].sum()

        # 결과 저장

        result_updown.append([up, down])

    return result_updown







def ensemble_check(a):
  buy = 0
  sell = 0
  print(a)
  for up, down in a:
    if up >= down:
      buy += 1
    else:
      sell += 1
  if buy > sell:
    return 1
  else:
    return 0




In [ ]:
!pip install backtrader

In [ ]:
import backtrader as bt
import yfinance as yf
from datetime import datetime
import numpy as np
import pandas as pd
from itertools import product

In [ ]:


class CCIStrategy(bt.Strategy):
    params = (
        ('cci_period', 18),     # CCI의 기본 기간
        ('buy_threshold', 50),  # 매수 임계값
        ('sell_threshold', 70), # 매도 임계값
        ('add_inc_lower', 0.05),
        ('add_inc_upper', 0.05),
        ('add_buy_drop', 0.05), # 추가 매수 조건 (10% 하락)
    )

    def __init__(self):
        self.cci = bt.indicators.CommodityChannelIndex(self.data, period=self.params.cci_period)
        self.last_cci = 0
        self.price_profit = []
        self.trade_dates = []  # 매수/매도 날짜 기록
        # Initialize a flag to track if the condition is met

        self.past_data = []

    def next(self):
        date = self.data.datetime.date(0)  # 현재 데이터의 날짜
        cash = self.broker.get_cash()      # 현재 가용 자금
        price = self.data.close[0]         # 현재 종가
        size = int(cash / price)           # 살 수 있는 최대 주식 수
        self.past_data.append(self.cci[0])
        #print(f"condition : {self.cci[0]}, {self.cci[0] - self.last_cci}")

        if len(self.past_data) >= 8:
          updown = Probabilistic_search(self.past_data[-1], self.past_data[-2], self.past_data[-3], self.past_data[-4], self.past_data[-5], self.past_data[-6], self.past_data[-7], self.past_data[-8])
          signal = ensemble_check(updown)

          # Buy condition and find_sell_condition(self.cci[0], self.cci[0] - self.last_cci) == 0
          if not self.position and signal == 1:
              self.buy(size=size)
              #print(f"buy cci : {self.cci[0]}, price : {price}")
              self.trade_dates.append({"action": "buy", "date": date, "price": price})
          # Sell condition
          elif self.position and signal == 0:
              profit = price - self.position.price
              self.sell(size=self.position.size)
              self.price_profit.append(profit)
              #print(f"1매수 가격: {self.position.price}, 매도 가격: {price}, 수익: {profit}")
              self.trade_dates.append({"action": "sell", "date": date, "price": price})

        self.last_cci = self.cci[0]

    def stop(self):
        if self.position:
            price = self.data.close[0]
            date = self.data.datetime.date(0)
            profit = price - self.position.price
            self.sell(size=self.position.size)
            self.price_profit.append(profit)
            #print(f"2매수 가격: {self.position.price}, 매도 가격: {price}, 수익: {profit}")
            self.trade_dates.append({"action": "final_sell", "date": date, "price": price})

# Fetch data from Yahoo Finance using yfinance
def fetch_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    data.columns = ['open', 'high', 'low', 'close', 'volume']
    return data

# US stock symbols
us_stock_symbols = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "ADBE",
    "ORCL", "INTC", "JPM", "BAC", "C", "GS", "MS", "JNJ", "PFE", "UNH", "KO", "DIS",
    "V", "MA", "HD", "WMT", "PG", "XOM", "CVX", "PEP", "NKE", "COST",
    "NFLX", "T", "VZ", "BA", "MRK", "ABBV", "AMGN", "CRM", "QCOM", "IBM", "BITX"
]

us_stock_symbols = [
    "AAPL", "NVDA"
]

all_trades = []  # 통합 데이터를 저장할 리스트

for symbol in us_stock_symbols:
    start_date = "2021-01-01"
    end_date = "2024-11-16"
    df = fetch_data(symbol, start=start_date, end=end_date)
    data = bt.feeds.PandasData(dataname=df)

    cerebro = bt.Cerebro()

    # Use a container to hold the strategy instance
    strategy_container = {}

    def strategy_creator(*args, **kwargs):
        strategy_container['instance'] = CCIStrategy(*args, **kwargs)
        return strategy_container['instance']

    cerebro.addstrategy(strategy_creator)
    cerebro.adddata(data)
    cerebro.broker.setcash(100000)
    cerebro.run()

    # Access the strategy instance after running cerebro
    strategy_instance = strategy_container.get('instance')
    if strategy_instance is None:
        continue

    trade_date = strategy_instance.trade_dates

    # Save all trades to a CSV file
    trade_date_df = pd.DataFrame(trade_date)
    output_file = symbol + "_trades.csv"
    trade_date_df.to_csv(output_file, index=False)

    profits = strategy_instance.price_profit
    wins = len([p for p in profits if p > 0])
    losses = len([p for p in profits if p <= 0])
    win_rate = (wins / len(profits) * 100) if profits else 0
    average_profit = sum([p for p in profits if p > 0]) / wins if wins > 0 else 0
    average_loss = sum([p for p in profits if p <= 0]) / losses if losses > 0 else 0
    #print(f"Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")
    all_trades.append({
        "symbol": symbol, "Total Trades": len(profits), "Winning Trades": wins, "Losing Trades": losses,
        "Win Rate": win_rate, "Average Profit": average_profit, "Average Loss": average_loss, "total_money" : cerebro.broker.getvalue()
    })


# Save all trades to a CSV file
all_trades_df = pd.DataFrame(all_trades)
output_file = "all_trades.csv"
all_trades_df.to_csv(output_file, index=False)


[*********************100%***********************]  1 of 1 completed


[[20, 16], [11, 9], [42, 12], [1, 2], [7, 0], [2, 0], [3, 0]]
[[38, 21], [10, 17], [8, 11], [14, 8], [2, 1], [2, 2], [1, 0]]
[[23, 12], [20, 13], [9, 12], [4, 11], [9, 11], [1, 4], [3, 2]]
[[15, 2], [13, 10], [11, 12], [9, 5], [13, 5], [9, 8], [1, 3]]
[[30, 20], [4, 6], [15, 8], [4, 17], [6, 3], [2, 8], [10, 6]]
[[24, 13], [22, 15], [8, 4], [8, 7], [8, 14], [4, 5], [6, 9]]
[[21, 15], [14, 6], [37, 17], [7, 8], [9, 10], [9, 10], [4, 8]]
[[11, 11], [8, 13], [7, 10], [39, 13], [3, 9], [7, 8], [10, 8]]
[[14, 5], [7, 5], [7, 7], [12, 7], [27, 11], [3, 7], [4, 13]]
[[17, 20], [10, 6], [4, 5], [6, 7], [2, 7], [9, 4], [2, 5]]
[[11, 10], [16, 7], [4, 8], [6, 3], [5, 3], [0, 3], [9, 9]]
[[19, 14], [8, 9], [6, 8], [6, 3], [2, 5], [6, 3], [1, 3]]
[[15, 9], [15, 10], [5, 5], [7, 9], [2, 2], [2, 6], [6, 1]]
[[10, 14], [6, 6], [5, 8], [5, 7], [6, 2], [6, 5], [2, 4]]
[[4, 6], [4, 8], [5, 3], [8, 5], [2, 5], [2, 2], [4, 3]]
[[18, 13], [2, 6], [5, 8], [4, 4], [9, 3], [2, 5], [1, 3]]
[[18, 13], [9, 8], [

[*********************100%***********************]  1 of 1 completed


[[10, 7], [4, 3], [6, 14], [31, 10], [10, 8], [2, 0], [7, 8]]
[[19, 8], [8, 6], [4, 2], [12, 7], [10, 6], [6, 7], [2, 3]]
[[19, 8], [13, 5], [4, 8], [1, 2], [10, 9], [7, 8], [8, 1]]
[[10, 6], [13, 5], [7, 3], [5, 6], [5, 7], [8, 6], [5, 7]]
[[36, 23], [7, 2], [7, 3], [7, 4], [3, 5], [5, 5], [4, 3]]
[[28, 13], [24, 12], [7, 7], [7, 4], [3, 1], [4, 6], [5, 5]]
[[51, 15], [13, 8], [13, 8], [2, 4], [3, 1], [1, 3], [4, 5]]
[[24, 13], [58, 19], [10, 6], [9, 11], [6, 5], [1, 3], [3, 6]]
[[15, 9], [14, 6], [48, 9], [8, 1], [7, 7], [2, 3], [3, 6]]
[[15, 12], [6, 6], [7, 10], [28, 9], [4, 2], [4, 8], [5, 2]]
[[11, 13], [6, 10], [5, 3], [12, 7], [6, 5], [4, 4], [7, 7]]
[[11, 11], [4, 9], [5, 4], [4, 4], [2, 7], [10, 3], [5, 4]]
[[12, 11], [8, 3], [5, 4], [3, 4], [3, 1], [0, 3], [2, 8]]
[[14, 13], [10, 9], [5, 3], [6, 7], [4, 6], [1, 4], [1, 3]]
[[5, 14], [6, 11], [5, 3], [4, 2], [5, 2], [6, 2], [2, 1]]
[[6, 19], [11, 6], [3, 5], [3, 5], [5, 3], [3, 4], [8, 3]]
[[18, 10], [5, 10], [7, 5], [6, 4], 

# 개선 및 정리


In [ ]:
import backtrader as bt
import pandas as pd
import yfinance as yf


# Probabilistic search function
def Probabilistic_search(cci, cci1, cci2, cci3, cci4, cci5, cci6, cci7):
    state = label_state(cci, cci_bins)
    state1 = label_state(cci1, cci_bins)
    state2 = label_state(cci2, cci_bins)
    state3 = label_state(cci3, cci_bins)
    state4 = label_state(cci4, cci_bins)
    state5 = label_state(cci5, cci_bins)
    state6 = label_state(cci6, cci_bins)
    last_state = label_state(cci7, cci_bins)

    state_list = [state, state1, state2, state3, state4, state5, state6, last_state]
    matrix_list = [matrix, matrix1, matrix2, matrix3, matrix4, matrix5, matrix6]

    result_updown = []

    for i in range(len(state_list) - 1):
        index = f"State{'_' + str(i) if i else ''}"
        scope_index = f"scope_label{'_' + str(i) if i else ''}"

        current_state = state_list[i]
        next_state = state_list[i + 1]
        scope = float(label_state_scope(current_state - next_state, cci_scope_bins))

        current_matrix = matrix_list[i]
        filter_matrix = current_matrix.loc[
            (current_matrix[index] == str(float(current_state))) &
            (current_matrix[scope_index] == scope)
        ].copy()

        filter_matrix["Next_State"] = filter_matrix["Next_State"].astype(float)

        down = filter_matrix.loc[
            (filter_matrix["Next_State"] < int(current_state)) &
            (filter_matrix["Next_State"] > int(current_state) - 2)
        ]["actual_count"].sum()

        up = filter_matrix.loc[
            (filter_matrix["Next_State"] > int(current_state)) &
            (filter_matrix["Next_State"] < int(current_state) + 2)
        ]["actual_count"].sum()

        result_updown.append([up, down])

    return result_updown


# Ensemble check function
def ensemble_check(a):
    buy = sum(1 for up, down in a if up >= down)
    sell = len(a) - buy
    return 1 if buy > sell else 0


# Custom Backtrader strategy
class CCIStrategy(bt.Strategy):
    params = (
        ('cci_period', 18),
        ('buy_threshold', 50),
        ('sell_threshold', 70),
        ('add_inc_lower', 0.05),
        ('add_inc_upper', 0.05),
        ('add_buy_drop', 0.05),
    )

    def __init__(self):
        self.cci = bt.indicators.CommodityChannelIndex(self.data, period=self.params.cci_period)
        self.last_cci = 0
        self.price_profit = []
        self.trade_dates = []
        self.past_data = []

    def next(self):
        date = self.data.datetime.date(0)
        cash = self.broker.get_cash()
        price = self.data.close[0]
        size = int(cash / price)
        self.past_data.append(self.cci[0])

        if len(self.past_data) >= 8:
            updown = Probabilistic_search(
                self.past_data[-1], self.past_data[-2], self.past_data[-3], self.past_data[-4],
                self.past_data[-5], self.past_data[-6], self.past_data[-7], self.past_data[-8]
            )
            signal = ensemble_check(updown)

            if not self.position and signal == 1:
                self.buy(size=size)
                self.trade_dates.append({"action": "buy", "date": date, "price": price})
            elif self.position and signal == 0:
                profit = price - self.position.price
                self.sell(size=self.position.size)
                self.price_profit.append(profit)
                self.trade_dates.append({"action": "sell", "date": date, "price": price})

        self.last_cci = self.cci[0]

    def stop(self):
        if self.position:
            price = self.data.close[0]
            date = self.data.datetime.date(0)
            profit = price - self.position.price
            self.sell(size=self.position.size)
            self.price_profit.append(profit)
            self.trade_dates.append({"action": "final_sell", "date": date, "price": price})


# Fetch data function
def fetch_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    data.columns = ['open', 'high', 'low', 'close', 'volume']
    return data


# Main trading loop
us_stock_symbols = ["NVDA"]
all_trades = []

for symbol in us_stock_symbols:
    start_date = "2021-01-01"
    end_date = "2024-11-16"
    df = fetch_data(symbol, start=start_date, end=end_date)
    data = bt.feeds.PandasData(dataname=df)

    cerebro = bt.Cerebro()
    strategy_container = {}

    def strategy_creator(*args, **kwargs):
        strategy_container['instance'] = CCIStrategy(*args, **kwargs)
        return strategy_container['instance']

    cerebro.addstrategy(strategy_creator)
    cerebro.adddata(data)
    cerebro.broker.setcash(100000)
    cerebro.run()

    strategy_instance = strategy_container.get('instance')
    if strategy_instance is None:
        continue

    trade_date = strategy_instance.trade_dates
    trade_date_df = pd.DataFrame(trade_date)
    trade_date_df.to_csv(symbol + "_trades.csv", index=False)

    profits = strategy_instance.price_profit
    wins = len([p for p in profits if p > 0])
    losses = len([p for p in profits if p <= 0])
    win_rate = (wins / len(profits) * 100) if profits else 0
    average_profit = sum([p for p in profits if p > 0]) / wins if wins > 0 else 0
    average_loss = sum([p for p in profits if p <= 0]) / losses if losses > 0 else 0

    all_trades.append({
        "symbol": symbol, "Total Trades": len(profits), "Winning Trades": wins,
        "Losing Trades": losses, "Win Rate": win_rate, "Average Profit": average_profit,
        "Average Loss": average_loss, "total_money": cerebro.broker.getvalue()
    })

all_trades_df = pd.DataFrame(all_trades)
all_trades_df.to_csv("all_trades.csv", index=False)


[*********************100%***********************]  1 of 1 completed


KeyError: 'scope_label_1'